# 自举聚合与随机森林

In [1]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split

# 创建随机数据集
X, y = make_classification(
    n_samples=1000,  # 数据集大小
    n_features=16,  # 特征数，及数据维度
    n_informative=5,  # 有效特征数
    n_redundant=2,  # 冗余特征数
    n_classes=2,  # 类别数
    flip_y=0.1,  # 类别随机的样本个数，该值越大，分类越困难
    random_state=0  # 随机种子
)
print(X.shape)

(1000, 16)


In [ ]:
class RandomForest:
    def __init__(self, n_trees=10, max_features='sqrt'):
        # max_feature是DTC的参数，表示节点分裂时随机采样的特征个数
        # sqrt代表取全部特征的平方根，None：取全部特征，log2：取全部特征的对数
        self.n_trees = n_trees
        self.oob_score = 0
        self.trees = [DTC(max_features=max_features) for _ in range(n_trees)]

    # 用X和y训练模型
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.n_classes = np.unique(y).shape[0]
        